### Impordid

In [55]:
import numpy as np
import pandas as pd

# video
import cv2 as cv

# kõnesüntees
# from gtts import gTTS
import IPython.display as ipd

# vb ajutine
import matplotlib.pyplot as plt

# mudel
import tensorflow as tf
from tensorflow import keras
import pathlib
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Rescaling, MaxPooling2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

### Mudelite ettevalmistus

In [22]:
train_kaust = pathlib.Path('asl_alphabet_train/')

test_kaust = pathlib.Path('asl_alphabet_test/')

In [23]:
batch_size = 32
img_height = 200
img_width = 200

train = tf.keras.utils.image_dataset_from_directory(
    train_kaust,
    labels='inferred',
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="training",
    image_size=(img_height, img_width),
    batch_size=batch_size)

# test kaustas on ainult 1 pilt iga viipe kohta, seega seda ei saa kasutada valideerimiseks
val = tf.keras.utils.image_dataset_from_directory(
    train_kaust,
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="validation",
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 81000 files belonging to 27 classes.
Using 64800 files for training.
Found 81000 files belonging to 27 classes.
Using 16200 files for validation.


### Mudeli treenimine

In [112]:
mudel4 = Sequential()
mudel4.add(Rescaling(1./255, input_shape=(img_height, img_width, 3)))

mudel4.add(Conv2D(16, 3, padding="same", activation="relu"))
mudel4.add(BatchNormalization())
mudel4.add(MaxPooling2D())
mudel4.add(Conv2D(32, 3, padding="same", activation="relu"))
mudel4.add(BatchNormalization())
mudel4.add(MaxPooling2D())
mudel4.add(Conv2D(64, 3, padding="same", activation="relu"))
mudel4.add(Dropout(0.25))
mudel4.add(BatchNormalization())
mudel4.add(MaxPooling2D())

mudel4.add(Flatten())
mudel4.add(Dense(256, activation='relu'))
mudel4.add(Dense(27, activation='softmax'))

In [113]:
mudel4.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [114]:
mudel4.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_26 (Rescaling)    (None, 200, 200, 3)       0         
                                                                 
 conv2d_62 (Conv2D)          (None, 200, 200, 16)      448       
                                                                 
 batch_normalization_42 (Bat  (None, 200, 200, 16)     64        
 chNormalization)                                                
                                                                 
 max_pooling2d_60 (MaxPoolin  (None, 100, 100, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_63 (Conv2D)          (None, 100, 100, 32)      4640      
                                                                 
 batch_normalization_43 (Bat  (None, 100, 100, 32)   

In [ ]:
history = mudel4.fit(train, epochs=5, validation_data=val)

Epoch 1/5
2025/2025 [==============================] - 1443s 712ms/step - loss: 2.3486 - accuracy: 0.2977 - val_loss: 1.5036 - val_accuracy: 0.5007
Epoch 2/5
2025/2025 [==============================] - 1473s 727ms/step - loss: 0.6341 - accuracy: 0.7799 - val_loss: 0.6123 - val_accuracy: 0.7837
Epoch 3/5
2025/2025 [==============================] - 1620s 800ms/step - loss: 0.3216 - accuracy: 0.8907 - val_loss: 0.5076 - val_accuracy: 0.8523
Epoch 4/5
2025/2025 [==============================] - 1517s 749ms/step - loss: 0.2162 - accuracy: 0.9288 - val_loss: 1.5568 - val_accuracy: 0.7227
Epoch 5/5
 370/2025 [====>.........................] - ETA: 18:46 - loss: 0.1811 - accuracy: 0.9418

In [ ]:
# salvestamine
mudel4.save('mudel.h5')

In [ ]:
# sisse lugemiseks
mudel4 = tf.keras.models.load_model('mudel.h5')